# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


<div class="alert alert-block alert-info">
 Пропуски в данных отсутствуют. Также видно, что перед нами классическая задача классификации(предсказываем, есть ли страховые выплаты или нет). Наличие нулей в "Члены семьи" свидетельствует о том, что отсчет идет с нуля.(0 - 1 человек и т.д)
</div>

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Нет. Качество не изменится.

**Обоснование:** 

${Доказательство}
$
Умножение матрицы признаков на некоторую обратимую матрицу(очевидно, совпадающей размерности) будем обозначать следующим образом:$XP$ 

Подставим в формулу 

$$
a = Xw
$$

вместо w 
$$
(X^T X)^{-1} X^T y
$$
Получим:
$$
a=X(X^TX)^{-1}X^Ty
$$
Требуется умножать матрицу признаков X на обратимую матрицу P. Заменим все X на XP:
$$
a_1=XP((XP)^TXP)^{-1}(XP)^Ty
$$
Из университетского курса линейной алгебры известны следующие свойства умножения матриц:
$$
(AB)^{-1}=B^{-1}A^{-1}
$$
$$
(AB)^{T}=B^{T}A^{T}
$$
Используя это свойство, раскроем скобки:

$$
a_1= XP(XP)^{-1} ((XP)^T)^{-1} P^T X^Ty
$$



$$
a_1=XPP^{-1}X^{-1} (X^T)^{-1} (P^T)^{-1} P^TX^Ty
$$


Исправленный вариант:
Вернемся к состоянию 
$$
a_1= XP(XP)^{-1} ((XP)^T)^{-1} P^T X^Ty
$$
Произведение $XP(XP)^{-1}$ дает единичную матрицу:
$$
a_1= E((XP)^T)^{-1} P^T X^Ty
$$
Свернем обратно $P^T X^T$:
$$
a_1= E((XP)^T)^{-1} (XP)^Ty
$$
Произведение $((XP)^T)^{-1} (XP)^T$ дает единичную матрицу:
$$
a_1= Ey
$$
Отсюда $a_1=y=a$

$$
a_1=XP((XP)^TXP)^{-1}(XP)^Ty
$$

$$
a_1=XP(XP)^{-1}((XP)^T)^{-1}(XP)^Ty
$$

$$
a_1=XPP^{-1}X^{-1}((XP)^T)^{-1}(XP)^Ty
$$



$$
a_1=XEX^{-1}((XP)^T)^{-1}(XP)^Ty
$$


$$
a_1=XX^{-1}(P^TX^T)^{-1}P^TX^Ty=XX^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^Ty
$$


$$
a_1=XX^{-1}(X^T)^{-1}EX^Ty=X(X^TX)^{-1}X^Ty=Xw=a
$$


$$
a' = XP((XP)^T XP)^{-1} (XP)^T y
$$
$$
a' = XP(P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
a' = XP(P^T (X^T X) P)^{-1} P^T X^T y
$$

$$
a' = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$
a' = X(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$
a' = X(X^TX)^{-1}X^Ty
$$
Т.к
$$
w=(X^TX)^{-1} X^Ty
$$
Получаем
$$
Xw=a'
$$

Укажем, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

In [4]:
features=df.drop('Страховые выплаты',axis=1)
target=df['Страховые выплаты']

In [5]:
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                            test_size=0.25, random_state=12345)

In [6]:
model=LinearRegression()
model.fit(features_train,target_train)

LinearRegression()

In [7]:
predictions=model.predict(features_test)

Метрика R2

In [8]:
print(r2_score(target_test,predictions))

0.43522757127026546


По формуле 
$$
(X^TX)^{-1}X^Ty
$$

In [9]:
X = np.concatenate((np.ones((features_train.shape[0], 1)), features_train), axis=1)
y = target_train
w = np.linalg.inv(X.T.dot(X))@ X.T @ y 

In [10]:
print(w[1:])

[ 1.79258369e-02  3.57228278e-02 -5.46000708e-07 -1.26186590e-02]


In [11]:
model.coef_

array([ 1.79258369e-02,  3.57228278e-02, -5.46000708e-07, -1.26186590e-02])

Коэффициенты регрессии совпадают.

## Алгоритм преобразования

**Алгоритм**

Выше было выяснено, что умножение матрицы признаков на некоторую обратимую матрицу не ухудшит качество регрессии. Поэтому, на этапе поиска коэффициентов, умножим X на P. P будем генерировать(в курсе было сказано, что вероятность получить необратимую матрицу практически равна нулю).

**Обоснование**

Качество линеной регрессии не изменится, ввиду проведенных выше теоретических выкладок(пункт 2)

## Проверка алгоритма

Поделил выборки на две части(train и test), дабы потом умножить каждую из них на матрицу P

In [12]:
X_train=features_train.values
print(X.shape)

(3750, 5)


In [13]:
X_test=features_test.values

Матрица должна быть квадратной, а значит будет размерности 4x4(дабы совпадать с количеством признаков и была возможность умножать матрицы)

In [21]:
P = [[random.randrange(0,10) for y in range(4)] for x in range(4)]

In [22]:
determinant=np.linalg.det(P)
print(determinant)
if determinant==0:
    P = [[random.randrange(0,10) for y in range(4)] for x in range(4)]
    determinant=np.linalg.det(P)
    print(determinant)

422.99999999999983


In [15]:
print(P)


[[2, 2, 9, 7], [2, 8, 2, 1], [0, 4, 0, 0], [9, 7, 5, 7]]


In [17]:
Z_train=X_train @ P
print(Z_train)

[[9.70000e+01 1.45153e+05 1.00000e+02 5.70000e+01]
 [7.00000e+01 2.30674e+05 7.70000e+01 4.10000e+01]
 [7.30000e+01 1.64663e+05 6.90000e+01 3.90000e+01]
 ...
 [9.30000e+01 1.79137e+05 9.60000e+01 5.50000e+01]
 [8.00000e+01 2.00604e+05 6.40000e+01 5.00000e+01]
 [3.80000e+01 1.63352e+05 3.80000e+01 1.90000e+01]]


In [ ]:
Z_test=X_test @ P

In [ ]:
print(Z_test.shape)

In [ ]:
print(Z_train.shape)

Теперь обучимся на этих данных:

In [ ]:
model_Z=LinearRegression()
model_Z.fit(Z_train, target_train)

In [ ]:
predictions_z=model_Z.predict(Z_test)

In [ ]:
print(r2_score(target_test,predictions))

Значение, полученное до Z-преобразования было 0.43522757127026546. Отличий действительно нет

## Вывод

В ходе выполнения проекта было доказано, что умножение матрицы признаков на случайную обратимую не ухудшает качество модели. Этот факт может позволить преобразовывать данные и, тем самым, делать их защищенными от третьих лиц. В ходе выполнения проекта этот алгоритм был испытан и доказал свою работоспособность и отсутствие ухудшения качества. Для усиления защиты можно генерировать элементы новой матрицы P(обратимой) с большим разбросом значений(все равно метрика R2 останется неизменной)

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.